Assignment 5 \
Part 4: DT-Credit Regression \
Submitted by Group 5 \

Groupmembers:

1. Sajan Kumer Sarker (2111131642) (sajan.sarker@northsouth.edu)
2. Rafsan Jani Chowdhury (2011424642) (rafsan.chowdhury@northsouth.edu)
3. Rosely Mohammad (2014219642) (rosely.mohammad@northsouth.edu)

Task: \
1) Support vector regression (SVR)


**For Regression problem: Mean-squared error**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from collections import defaultdict

In [3]:
df = pd.read_csv('/content/drive/MyDrive/CSE445-Assignment/dataset/DT-Credit.csv')
print(df.shape)
print(df.head())

(400, 11)
    Income  Limit  Rating  Cards  Age  Education  Own Student Married Region  \
0   14.891   3606     283      2   34         11   No      No     Yes  South   
1  106.025   6645     483      3   82         15  Yes     Yes     Yes   West   
2  104.593   7075     514      4   71         11   No      No      No   West   
3  148.924   9504     681      3   36         11  Yes      No      No   West   
4   55.882   4897     357      2   68         16   No      No     Yes  South   

   Balance  
0      333  
1      903  
2      580  
3      964  
4      331  


In [4]:
print('Number of missing values: ')
print(df.isnull().sum())

Number of missing values: 
Income       0
Limit        0
Rating       0
Cards        0
Age          0
Education    0
Own          0
Student      0
Married      0
Region       0
Balance      0
dtype: int64


In [5]:
df = pd.get_dummies(df, columns=['Own', 'Student', 'Married', 'Region'])
print(df.shape)
print(df.head())

(400, 16)
    Income  Limit  Rating  Cards  Age  Education  Balance  Own_No  Own_Yes  \
0   14.891   3606     283      2   34         11      333    True    False   
1  106.025   6645     483      3   82         15      903   False     True   
2  104.593   7075     514      4   71         11      580    True    False   
3  148.924   9504     681      3   36         11      964   False     True   
4   55.882   4897     357      2   68         16      331    True    False   

   Student_No  Student_Yes  Married_No  Married_Yes  Region_East  \
0        True        False       False         True        False   
1       False         True       False         True        False   
2        True        False        True        False        False   
3        True        False        True        False        False   
4        True        False       False         True        False   

   Region_South  Region_West  
0          True        False  
1         False         True  
2         False    

In [6]:
X = df.drop(columns=['Balance'])
y = df['Balance']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f'Train Data Shape (X, y): {X_train.shape, y_train.shape}')
print(f'Validation Data Shape (X, y): {X_val.shape, y_val.shape}')
print(f'Test Data Shape (X, y): {X_test.shape, y_test.shape}')

Train Data Shape (X, y): ((280, 15), (280,))
Validation Data Shape (X, y): ((60, 15), (60,))
Test Data Shape (X, y): ((60, 15), (60,))


In [7]:
# mse calculation
def mse(y_actual, y_pred):
    error = 0.0

    for actual, predicted in zip(y_actual, y_pred):
        error += (actual - predicted) ** 2
    mse = error / len(y_actual)
    return mse

In [8]:
# SVM base model
svm_base = SVR()
svm_base.fit(X_train, y_train)

y_val_pred = svm_base.predict(X_val)
print('Validation Set MSE:', mse(y_val, y_val_pred))

Validation Set MSE: 190887.40728903702


In [10]:
param_grid = {
    'C': [0.1, 1, 10],
    'epsilon': [0.01, 0.1, 0.5],
    'kernel': ['linear', 'rbf', 'poly']
}

svr = SVR()
svm_grid_search = GridSearchCV(estimator=svr, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
svm_grid_search.fit(X_train, y_train)

best_params = svm_grid_search.best_params_
print("Best Parameters:", best_params)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Best Parameters: {'C': 10, 'epsilon': 0.1, 'kernel': 'linear'}


In [11]:
svm_final = SVR(**best_params)
svm_final.fit(X_train, y_train)

y_val_pred = svm_final.predict(X_val)
print('Validation Set MSE:', mse(y_val, y_val_pred))

Validation Set MSE: 11336.466847405549


In [12]:
# now retrain the model on the combined training and validation set with the best hyperparameters
X_train_val = np.concatenate((X_train, X_val))
y_train_val = np.concatenate((y_train, y_val))

svm_final.fit(X_train_val, y_train_val)

y_test_pred = svm_final.predict(X_test)
print('Test Set MSE:', mse(y_test, y_test_pred))

Test Set MSE: 13779.732830605468
